<a href="https://colab.research.google.com/github/DriGutierrez/MarketWiseAI/blob/main/Projeto_MarketWiseAI_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes_(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [17]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [18]:
# Configura o cliente da SDK do Gemini

from google import genai
import os # Importa o módulo os para acessar variáveis de ambiente

# Recupera a chave da API da variável de ambiente e a armazena em uma variável Python
GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')

# Use o argumento api_key para passar a chave
client = genai.Client(api_key=GOOGLE_API_KEY)

MODEL_ID = "gemini-2.0-flash"

In [22]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo qual o nome do novo papa ###############################################
resposta = client.models.generate_content(model=MODEL_ID, contents='Qual o nome do novo papa?',)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 O Papa atual é o Papa Francisco. Ele foi eleito em 13 de março de 2013.


In [24]:
# Pergunta ao Gemini sobre o novo Papa UTILIZANDO A BUSCA DO GOOGLE

response_busca = client.models.generate_content(model=MODEL_ID, contents='Qual o nome do novo papa?', config={"tools": [{"google_search": {}}]})

# Exibe a resposta com a busca na tela
display(Markdown(f"Resposta (com busca):\n {response_busca.text}"))

# Exibe informações sobre a busca (opcional)
print(f"Busca realizada: {response_busca.candidates[0].grounding_metadata.web_search_queries}")
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response_busca.candidates[0].grounding_metadata.grounding_chunks])}")

Resposta (com busca):
 O novo papa é o Cardeal Robert Prevost, que adotou o nome de Leão XIV. Ele foi eleito em 8 de maio de 2025.


Busca realizada: ['novo papa']
Páginas utilizadas na resposta: youtube.com, forbes.com.br, youtube.com, youtube.com, youtube.com


In [26]:
# Exibe a busca
print(f"Busca realizada: {response_busca.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response_busca.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response_busca.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['novo papa']
Páginas utilizadas na resposta: youtube.com, forbes.com.br, youtube.com, youtube.com, youtube.com



In [27]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [28]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [29]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [30]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [65]:
# --- Configuração do Usuário ---
concorrentes = [
    "NomeDoSeuPrincipalConcorrente1",
    "NomeDeOutroConcorrenteRelevante",
    # Adicione mais concorrentes conforme necessário
]

topicos_tendencia = [
    "Tendências de marketing de luxo 2025",
    "Inovações em eventos de alto padrão",
    # Adicione mais tópicos de tendência conforme necessário
]

publico_alvo = "Empreendedores e profissionais de marketing de alto padrão" # Descreva seu público-alvo principal

fontes_especialistas = [
    "Publicações líderes em marketing de luxo",
    "Principais influenciadores e especialistas em eventos exclusivos",
    # Adicione fontes de informação de especialistas relevantes para o seu setor
]

data_de_hoje = date.today().strftime("%d/%m/%Y")

In [66]:
##########################################
# --- Agente 1: Buscador de Tendências --- #
##########################################
def agente_buscador(concorrentes, topicos_tendencia, publico_alvo, fontes_especialistas, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um assistente de pesquisa avançado especializado em inteligência competitiva, análise de tendências e compreensão do público-alvo.
        Sua tarefa é usar a ferramenta de busca do Google (google_search) para recuperar informações relevantes sobre:

        1. As estratégias de marketing, eventos e comunicação dos seguintes concorrentes: {', '.join(concorrentes)}.
        2. As últimas tendências e novidades relacionadas aos seguintes tópicos: {', '.join(topicos_tendencia)}, buscando insights de especialistas.
        3. As principais necessidades, desejos, desafios e comportamentos online do público-alvo: {publico_alvo}.
        4. As perspectivas e previsões de fontes de informação de especialistas relevantes: {', '.join(fontes_especialistas)}.

        Foque em informações recentes (últimos meses) que possam fornecer insights sobre as abordagens dos concorrentes, as direções do mercado apontadas por especialistas e o que motiva o público-alvo.
        Tente encontrar dados como campanhas recentes, lançamentos, posicionamento de marca, relatórios de tendências, artigos de especialistas e discussões sobre o público-alvo.
        """,
        description="Agente que busca informações abrangentes sobre concorrentes, tendências e público-alvo no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"""
    Concorrentes a serem pesquisados: {', '.join(concorrentes)}
    Tópicos de tendência a serem pesquisados: {', '.join(topicos_tendencia)}
    Informações sobre o público-alvo: {publico_alvo}
    Fontes de informação de especialistas a serem consultadas: {', '.join(fontes_especialistas)}
    Data de hoje: {data_de_hoje}
    """

    resultados_busca = call_agent(buscador, entrada_do_agente_buscador)
    return resultados_busca

In [67]:
################################################
# --- Agente 2: Planejador de Estratégias --- #
################################################
def agente_planejador(topicos_tendencia, resultados_busca, publico_alvo):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um analista de inteligência de mercado sênior, especializado em integrar diversas fontes de informação para gerar insights estratégicos.
        Com base nos resultados da busca fornecidos, que contêm informações sobre concorrentes, tendências em: {', '.join(topicos_tendencia)}, e o público-alvo: {publico_alvo},
        sua tarefa é analisar essas informações de forma integrada para identificar:

        1. As principais estratégias e táticas dos concorrentes e como elas se alinham ou divergem das tendências de especialistas.
        2. As tendências mais relevantes identificadas por especialistas e como elas ressoam com as necessidades e comportamentos do público-alvo.
        3. Oportunidades de inovação e diferenciação que surgem da análise da concorrência, das tendências de especialistas e das necessidades do público.
        4. Possíveis lacunas no mercado que os concorrentes não estão explorando e que podem ser relevantes para o público-alvo, à luz das tendências.
        5. Riscos ou desafios potenciais identificados nas estratégias dos concorrentes ou nas tendências do mercado.

        Seja estratégico e orientado para a ação, fornecendo insights que um empreendedor pode usar para tomar decisões informadas e desenvolver um plano de ação eficaz.
        """,
        description="Agente que integra informações de busca sobre concorrentes, tendências e público-alvo para gerar insights estratégicos",
        tools=[] # O planejador não precisa de ferramentas externas no momento
    )

    entrada_do_agente_planejador = f"""
    Resultados da busca: {resultados_busca}
    Tópicos de tendência pesquisados: {', '.join(topicos_tendencia)}
    Informações sobre o público-alvo: {publico_alvo}
    """

    plano_de_analise_integrado = call_agent(planejador, entrada_do_agente_planejador)
    return plano_de_analise_integrado

In [68]:
######################################
# --- Agente 3: Relatórios --- #
######################################
def agente_relatorio(topicos_tendencia, plano_de_analise, concorrentes):
    relatorio = Agent(
        name="agente_relatorio",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um consultor estratégico de negócios com experiência em apresentar análises de mercado de forma clara e impactante.
        Com base na análise integrada fornecida sobre as estratégias de concorrentes ({', '.join(concorrentes)}) e as tendências em: {', '.join(topicos_tendencia)},
        sua tarefa é formatar essas informações em um relatório estratégico conciso e acionável para um empreendedor.

        O relatório deve incluir as seguintes seções principais:

        1.  **Visão Geral das Estratégias da Concorrência:** Resuma as principais abordagens e táticas observadas nos concorrentes. Destaque pontos fortes e fracos aparentes.
        2.  **Principais Tendências e Implicações:** Apresente as tendências mais relevantes identificadas por especialistas e discuta suas potenciais implicações para o mercado e para o negócio do empreendedor.
        3.  **Oportunidades e Lacunas:** Identifique oportunidades de inovação e diferenciação que surgem da análise, bem como possíveis lacunas que os concorrentes não estão explorando.
        4.  **Recomendações Estratégicas:** Forneça recomendações práticas e acionáveis para o empreendedor, com base nos insights gerados. Estas devem ser específicas e relevantes para o contexto do mercado analisado.

        Use uma linguagem profissional, clara e objetiva. Organize o relatório de forma lógica, com títulos e subtítulos para facilitar a leitura e a compreensão.
        """,
        description="Agente que formata a análise estratégica em um relatório acionável para empreendedores",
        tools=[] # O agente de relatório não precisa de ferramentas externas no momento
    )

    entrada_do_agente_relatorio = f"""
    Análise estratégica gerada: {plano_de_analise}
    Tópicos de tendência analisados: {', '.join(topicos_tendencia)}
    Concorrentes analisados: {', '.join(concorrentes)}
    """

    relatorio_final = call_agent(relatorio, entrada_do_agente_relatorio)
    return relatorio_final

In [69]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topicos_tendencia, plano_de_analise, relatorio_final, concorrentes):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um editor e revisor de conteúdo estratégico com experiência em relatórios de análise de mercado.
        Sua tarefa é revisar o relatório estratégico abaixo, que analisa as estratégias de concorrentes ({', '.join(concorrentes)}) e as tendências em: {', '.join(topicos_tendencia)},
        para garantir sua clareza, concisão, relevância e tom profissional.

        Verifique se o relatório:

        1.  Apresenta uma visão clara das estratégias da concorrência.
        2.  Destaca as principais tendências e suas implicações de forma compreensível.
        3.  Identifica oportunidades e lacunas de maneira lógica e fundamentada.
        4.  Fornece recomendações estratégicas que são práticas e relevantes para um empreendedor.
        5.  Utiliza uma linguagem profissional e objetiva, evitando jargões excessivos.
        6.  Está bem organizado e fácil de ler.

        Se o relatório estiver bom e pronto para ser apresentado, responda apenas 'O relatório está ótimo e pronto para apresentação!'.
        Caso haja problemas ou sugestões de melhoria, aponte-os de forma clara e construtiva.
        """,
        description="Agente revisor de relatórios de análise estratégica para empreendedores",
        tools=[] # O agente revisor não precisa de ferramentas externas no momento
    )

    entrada_do_agente_revisor = f"""
    Relatório a ser revisado: {relatorio_final}
    Análise estratégica que gerou o relatório: {plano_de_analise}
    Tópicos de tendência analisados: {', '.join(topicos_tendencia)}
    Concorrentes analisados: {', '.join(concorrentes)}
    """

    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
topico = input("❓ Por favor, digite o mercado sobre o qual você quer criar o relatório de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not concorrentes and not topicos_tendencia and not publico_alvo and not fontes_especialistas: print("Você não configurou as informações necessárias para a busca!")
else:
  print("🚀 Iniciando a análise abrangente de mercado 🚀")

  resultados_busca = agente_buscador(concorrentes, topicos_tendencia, publico_alvo, fontes_especialistas, data_de_hoje)
  print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(resultados_busca))
  print("--------------------------------------------------------------")

  plano_de_analise = agente_planejador(topicos_tendencia, resultados_busca, publico_alvo)
  print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_analise))
  print("--------------------------------------------------------------")

  relatorio_final = agente_relatorio(topicos_tendencia, plano_de_analise, concorrentes)
  print("\n--- 📝 Resultado do Agente 3 (Relatório) ---\n")
  display(to_markdown(relatorio_final))
  print("--------------------------------------------------------------")

  post_final = agente_revisor(topicos_tendencia, plano_de_analise, relatorio_final, concorrentes)
  print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))
  print("--------------------------------------------------------------")